In [1]:
import cv2
import numpy as np
import os
import shutil
import time
import csv

Generating Data

In [35]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img, scale_factor=1.05, min_neighbors= 10 , padding_height = 20):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbors)
        if len(faces) == 0:
            return None
        (x, y, w, h) = faces[0]
        # Calculate aspect ratio of the face
        aspect_ratio = float(w) / h
        # Calculate padding for height and width
        padding_height = max(0, int(padding_height))
        padding_width = int(padding_height * aspect_ratio)
        # Adding padding only to the height
        y = max(0, y - padding_height)
        h = min(img.shape[0], h + 2 * padding_height)
        # Adding padding to width to maintain aspect ratio
        x = max(0, x - padding_width)
        w = min(img.shape[1], w + 2 * padding_width)
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face

    cap = cv2.VideoCapture(0)
    id = 'Dat'
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/" + str(id) + "_" + str(img_id) + ".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)
            time.sleep(1)

            if cv2.waitKey(1) == 13 or int(img_id) == 1:
                break

    cap.release()
    cv2.destroyAllWindows()
    print("Thu thập dữ liệu thành công.......!")
    

In [36]:
generate_dataset()


Thu thập dữ liệu thành công.......!


Data Augmentaion

In [16]:
# Đường dẫn tới thư mục chứa ảnh gốc
input_dir = 'data/original_data/TuNam'
# Đường dẫn tới thư mục chứa ảnh đã tăng cường
output_dir = 'data/augmentation_data/TuNam'

# Tạo thư mục chứa ảnh đã tăng cường nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lấy danh sách các tập tin ảnh
image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# Hàm xoay ảnh
def rotate_image(image, angle):
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    return cv2.warpAffine(image, M, (cols, rows))

# Hàm dịch chuyển ảnh
def translate_image(image, x, y):
    M = np.float32([[1, 0, x], [0, 1, y]])
    return cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

# Hàm phóng to/thu nhỏ ảnh
def scale_image(image, fx, fy):
    return cv2.resize(image, None, fx=fx, fy=fy)

# Hàm lật ảnh
def flip_image(image, flip_code):
    return cv2.flip(image, flip_code)

# Hàm làm mờ ảnh
def blur_image(image, ksize):
    return cv2.GaussianBlur(image, (ksize, ksize), 0)

# Hàm thêm nhiễu vào ảnh
def add_noise(image, sigma):
    noise = np.random.normal(0, sigma, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

# Hàm thay đổi độ sáng của ảnh
def adjust_brightness(image, factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] = cv2.multiply(hsv[:, :, 2], factor)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# Hàm thay đổi độ tương phản của ảnh
def adjust_contrast(image, alpha):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=0)

# Hàm thay đổi độ bão hòa của ảnh
def adjust_saturation(image, factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 1] = cv2.multiply(hsv[:, :, 1], factor)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# Hàm Mixup
def mixup_images(image1, image2, alpha):
    return cv2.addWeighted(image1, alpha, image2, 1 - alpha, 0)

# Hàm CutMix
def cutmix_images(image1, image2, x, y, width, height):
    image1[y:y + height, x:x + width] = image2[y:y + height, x:x + width]
    return image1

# Hàm kiểm tra tên file
def is_valid_filename(filename):
    # Kiểm tra nếu tên file có dạng "Tên_stt.jpg"
    return filename.count('_') == 1 and filename.split('_')[0] != 'original'




In [3]:
for image_file in image_files:
    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)

    # Rotate image
    rotated_image = rotate_image(image, np.random.uniform(-25, 25))
    cv2.imwrite(os.path.join(output_dir, 'rotate_' + image_file), rotated_image)

    # Translate image
    translated_image = translate_image(image, np.random.uniform(-0.2 * image.shape[1], 0.2 * image.shape[1]), np.random.uniform(-0.2 * image.shape[0], 0.2 * image.shape[0]))
    cv2.imwrite(os.path.join(output_dir, 'translate_' + image_file), translated_image)

    # Scale image
    scaled_image = scale_image(image, np.random.uniform(0.8, 1.2), np.random.uniform(0.8, 1.2))
    cv2.imwrite(os.path.join(output_dir, 'scale_' + image_file), scaled_image)

    # Flip image
    flipped_image = flip_image(image, 1)
    cv2.imwrite(os.path.join(output_dir, 'flip_' + image_file), flipped_image)

    # Blur image
    blurred_image = blur_image(image, 5)
    cv2.imwrite(os.path.join(output_dir, 'blur_' + image_file), blurred_image)

    # Add noise to image
    noisy_image = add_noise(image, 25)
    cv2.imwrite(os.path.join(output_dir, 'noise_' + image_file), noisy_image)

    # Adjust brightness
    brightness_adjusted_image = adjust_brightness(image, np.random.uniform(0.5, 1.5))
    cv2.imwrite(os.path.join(output_dir, 'brightness_' + image_file), brightness_adjusted_image)

    # Adjust contrast
    contrast_adjusted_image = adjust_contrast(image, np.random.uniform(0.5, 1.5))
    cv2.imwrite(os.path.join(output_dir, 'contrast_' + image_file), contrast_adjusted_image)

    # Adjust saturation
    saturation_adjusted_image = adjust_saturation(image, np.random.uniform(0.5, 1.5))
    cv2.imwrite(os.path.join(output_dir, 'saturation_' + image_file), saturation_adjusted_image)

    # Mixup
    other_image_file = np.random.choice(image_files)
    other_image_path = os.path.join(input_dir, other_image_file)
    other_image = cv2.imread(other_image_path)
    mixup_image = mixup_images(image, other_image, np.random.uniform(0.2, 0.8))
    cv2.imwrite(os.path.join(output_dir, 'mixup_' + image_file), mixup_image)

    # CutMix
    cutmix_image_result = cutmix_images(image.copy(), other_image, np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0]), image.shape[1] // 4, image.shape[0] // 4)
    cv2.imwrite(os.path.join(output_dir, 'cutmix_' + image_file), cutmix_image_result)

print("Tăng cường dữ liệu đã hoàn thành.")

Tăng cường dữ liệu đã hoàn thành.


Data Combination

In [4]:
# Đường dẫn tới thư mục chứa ảnh
folder_path = 'data/original_data/TuNam'

# Lặp qua tất cả các file trong thư mục
for filename in os.listdir(folder_path):
    # Kiểm tra xem file có phải là file ảnh JPG không
    if filename.endswith('.jpg'):
        # Tách phần tên và phần mở rộng của file
        name, ext = os.path.splitext(filename)
        # Kiểm tra xem tên file đã có tiền tố 'original_' hay chưa
        if not name.startswith('original_'):
            # Tạo tên mới cho file
            new_name = f'original_{name}{ext}'
            # Đường dẫn đầy đủ tới file cũ và file mới
            old_file = os.path.join(folder_path, filename)
            new_file = os.path.join(folder_path, new_name)
            # Đổi tên file
            os.rename(old_file, new_file)
        else: 
            print("Đã có tiền tố original")

print("Đã đổi tên tất cả các file.")

Đã đổi tên tất cả các file.


In [5]:
def merge_folders(folder1, folder2, folder3, folder4, folder5, folder6, folder7, folder8, output_folder):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lấy danh sách tệp từ cả hai thư mục
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)
    files3 = os.listdir(folder3)
    files4 = os.listdir(folder4)
    files5 = os.listdir(folder5)
    files6 = os.listdir(folder6)
    files7 = os.listdir(folder7)
    files8 = os.listdir(folder8)



    # Ghép tất cả tệp từ folder1 vào thư mục đầu ra
    for file in files1:
        src_path = os.path.join(folder1, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)

    # Ghép tất cả tệp từ folder2 vào thư mục đầu ra
    for file in files2:
        src_path = os.path.join(folder2, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)

    for file in files3:
        src_path = os.path.join(folder3, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)
        
    for file in files4:
        src_path = os.path.join(folder4, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)

    for file in files5:
        src_path = os.path.join(folder5, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)
        
    for file in files6:
        src_path = os.path.join(folder6, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)
        
    for file in files7:
        src_path = os.path.join(folder7, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)
        
    for file in files8:
        src_path = os.path.join(folder8, file)
        dst_path = os.path.join(output_folder, file)
        shutil.copy(src_path, dst_path)
    


In [6]:
folder1 = "data/augmentation_data/Dat"
folder2 = "data/augmentation_data/Nam"
folder3 = "data/augmentation_data/NgocAnh"
folder4 = "data/augmentation_data/TuNam"
folder5 = "data/original_data/Dat"
folder6 = "data/original_data/Nam"
folder7 = "data/original_data/NgocAnh"
folder8 = "data/original_data/TuNam"
output_folder = "data/Final_data"

merge_folders(folder1, folder2, folder3, folder4, folder5,
              folder6, folder7, folder8, output_folder)

Data Labeling

In [7]:
# Dictionary để ánh xạ giữa tên file và nhãn
label_mapping = {
    '0': ['Dat'],
    '1': ['Nam'],
    '2': ['NgocAnh'],
    '3': ['TuNam']
}


def extract_name_from_filename(filename):
    parts = filename.split('_')
    # Lấy phần cuối cùng của danh sách parts (đó là phần tên)
    name = parts[1]
    return name


# Hàm để gắn nhãn cho ảnh
def label_images(folder_path):
    labeled_images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            # Trích xuất tên người từ tên file
            name = extract_name_from_filename(filename)
            # Kiểm tra xem tên người thuộc vào nhóm nào
            found_label = False
            for label, name_list in label_mapping.items():
                if name in name_list:
                    # Thêm thông tin về tên file và nhãn vào danh sách
                    labeled_images.append((filename, label))
                    found_label = True
                    break
            if not found_label:
                print(f"Không tìm thấy nhãn cho file: {filename}")
    return labeled_images


In [9]:
# Đường dẫn tới thư mục chứa các file ảnh
folder_path = 'data/Final_data'
# Đường dẫn tới file CSV để lưu kết quả
csv_file_path = 'labeled_images.csv'

# Gọi hàm label_images và lưu kết quả vào file CSV
labeled_images = label_images(folder_path)

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'Label'])
    for filename, label in labeled_images:
        writer.writerow([filename, label])

print(f"Đã lưu kết quả vào file CSV: {csv_file_path}")

Đã lưu kết quả vào file CSV: labeled_images.csv
